In [1]:
import numpy as np
import os
import glob
import pandas as pd
import tqdm

In [2]:
caption_files = glob.glob("data/captions/*.csv")

In [9]:
for caption_file in tqdm.tqdm(caption_files):
    x = np.array([]).reshape(0, 793 * 3 + 768 * 0)
    y = np.array([]).reshape(0, 84)
    
    video_id = caption_file.split("/")[-1].replace(".csv", "")
    text_feature_file = f"data/features/text/{video_id}.npy"

    captions = pd.read_csv(caption_file)

    text_features = np.load(text_feature_file)

    for i in range(len(captions)):
        t, d = captions.iloc[i, [0, 1]]

        audio_feature_file = f"data/features/audio/{video_id}/audio_feat_{t}_{d}.npy"
        audio_feature_d1_file = f"data/features/audio/{video_id}/audio_feat_d1_{t}_{d}.npy"
        audio_feature_d2_file = f"data/features/audio/{video_id}/audio_feat_d2_{t}_{d}.npy"
        hand_pose_file = f"data/features/handpose_openpose/{video_id}/handpose_{t}_{d}.npy"

        if os.path.exists(audio_feature_file):
            audio_features = np.load(audio_feature_file)
            audio_features_d1 = np.load(audio_feature_d1_file)
            audio_features_d2 = np.load(audio_feature_d2_file)
            
            if os.path.exists(hand_pose_file) and os.path.getsize(hand_pose_file) > 278:
                hand_poses = np.load(hand_pose_file, allow_pickle=True)
                
                hand_poses = hand_poses[:-1, :]
                
                x1 = audio_features.reshape(audio_features.shape[0], -1)
                x2 = audio_features_d1.reshape(audio_features_d1.shape[0], -1)
                x3 = audio_features_d2.reshape(audio_features_d2.shape[0], -1)
                # Add text features
                # x3 = text_features[i].reshape(1, -1).repeat(audio_features.shape[0], axis=0)
                
                xf = np.concatenate((x1, x2, x3), axis=1)
                
                x = np.concatenate((x, xf), axis=0)
                y = np.concatenate((y, hand_poses), axis=0)
    
    # Normalizing y
    lx, ly, rx, ry = y[:, :21], y[:, 21:42], y[:, 42:63], y[:, 63:]

    left_middle_finger_distance = np.sqrt(np.square(lx[:, 12]) + np.square(ly[:, 12]))
    scale_left = 1 / np.percentile(left_middle_finger_distance, 95)

    right_middle_finger_distance = np.sqrt(np.square(rx[:, 12]) + np.square(ry[:, 12]))
    scale_right = 1 / np.percentile(right_middle_finger_distance, 95)
    
    y = np.concatenate((lx * scale_left, ly * scale_left, rx * scale_right, ry * scale_right), axis=1)
                
    np.savez_compressed(f"data/train_data/openpose_mfcc_derivatives/{video_id}.npz", x=x, y=y)


100%|██████████| 58/58 [02:00<00:00,  2.07s/it]
